<a href="https://colab.research.google.com/github/niuhaoyu16/PythonforBeginners/blob/master/Decision_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_iris

In [2]:
from sklearn.tree import DecisionTreeClassifier

In [3]:
iris = load_iris()

In [4]:
X= iris.data[:, 2:]
y= iris.target

In [5]:
tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=2, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [7]:
from sklearn.tree import export_graphviz

In [8]:
export_graphviz(
    tree_clf,
    out_file="iris_tree.dot",
    feature_names = iris.feature_names[2:],
    class_names = iris.target_names,
    rounded = True,
    filled= True,
)

In [9]:
!dot -Tpdf iris_tree.dot -o iris_trees.pdf

In [10]:
 tree_clf.predict_proba([[5,1.5]])

array([[0.        , 0.90740741, 0.09259259]])

In [11]:
tree_clf.predict([[5,1.5]])

array([1])

In [12]:
from sklearn.tree import DecisionTreeRegressor


In [13]:
tree_reg = DecisionTreeRegressor(max_depth=2)

In [14]:
from sklearn.datasets import make_moons

In [15]:
X,y = make_moons(n_samples=10000, noise =0.4, random_state=42)

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=42)

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
params= {'max_leaf_nodes':list(range(2,100)), 'min_samples_split':[2,3,4]}


In [23]:
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1,
                           verbose=1, cv=3)

In [24]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 360 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 882 out of 882 | elapsed:    8.1s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
  

In [26]:
grid_search.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=17,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [27]:
from sklearn.metrics import accuracy_score

y_pred = grid_search.predict(X_test)
accuracy_score(y_test, y_pred)

0.8695

In [28]:
from sklearn.model_selection import ShuffleSplit

n_trees=1000
n_instances=100
mini_sets =[]

In [29]:
rs= ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)

In [30]:
rs

ShuffleSplit(n_splits=1000, random_state=42, test_size=7900, train_size=None)

In [31]:
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [35]:
from sklearn.base import clone
import numpy as np

In [36]:
forest = [clone(grid_search.best_estimator_) for _ in range(n_trees)]
accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
  tree.fit(X_mini_train, y_mini_train)

  y_pred = tree.predict(X_test)
  accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.8054499999999999

In [37]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

In [41]:
from scipy.stats import mode

In [43]:
y_pred_majority_votes, n_votes =mode(Y_pred, axis=0)

In [47]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.872